In [ ]:
!pip install rouge.score nltk py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install accelerate==0.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import GPT2LMHeadModel
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
max_input = 80
max_target = 80
batch_size = 64
model_checkpoints = "gpt2"

In [ ]:
data = load_dataset("bogdancazan/news-not-not-ela-text-simplification")
data

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 102711
    })
    test: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 1414
    })
})

In [ ]:
train_data = data['train'].to_pandas()
train_data['training'] = train_data['Normal']  + ' SIMPLE: ' + train_data['Simple']
train_data

,Normal,Simple,training
0,poll finds americans ok with women in combat,women get the ok to fight in combat units publ...,poll finds americans ok with women in combat S...
1,poll finds americans ok with women in combat,women get the ok to fight in combat units,poll finds americans ok with women in combat S...
2,women get the ok to fight in combat units publ...,women get the ok to fight in combat units,women get the ok to fight in combat units publ...
3,wars of the future will be fought with america...,american women will soon be free to fight on t...,wars of the future will be fought with america...
4,wars of the future will be fought with america...,american women will soon be able to fight on t...,wars of the future will be fought with america...
...,...,...,...
102706,its kernels are more spherical than other corn...,the thin layer or hull that surrounds the seed...,its kernels are more spherical than other corn...
102707,its kernels are more spherical than other corn...,there is a thin layer or hull that covers the ...,its kernels are more spherical than other corn...
102708,the thin layer or hull that surrounds the seed...,there is a thin layer or hull that covers the ...,the thin layer or hull that surrounds the seed...
102709,the thin layer or hull that surrounds the seed...,there is a thin layer that covers the seed.,the thin layer or hull that surrounds the seed...


In [ ]:
test_data = data['test'].to_pandas()
test_data['training'] = test_data['Normal']  + ' SIMPLE: ' + test_data['Simple']
test_data

,Normal,Simple,training
0,when the kernel is heated above degrees celsiu...,when the kernel gets very hot and the water in...,when the kernel is heated above degrees celsiu...
1,when the kernel is heated above degrees celsiu...,when the kernel gets very hot above degrees ce...,when the kernel is heated above degrees celsiu...
2,when the kernel is heated above degrees celsiu...,when the kernel gets very hot above degrees fa...,when the kernel is heated above degrees celsiu...
3,when the kernel gets very hot and the water in...,when the kernel gets very hot above degrees fa...,when the kernel gets very hot and the water in...
4,when the kernel gets very hot above degrees ce...,when the kernel gets very hot above degrees fa...,when the kernel gets very hot above degrees ce...
...,...,...,...
1409,this week u.s. ambassador to colombia kevin wh...,this week u.s. ambassador to colombia kevin wh...,this week u.s. ambassador to colombia kevin wh...
1410,and the country is on track to produce tons th...,colombian land dedicated to avocado production...,and the country is on track to produce tons th...
1411,colombian land dedicated to avocado production...,the country is on track to produce tons this y...,colombian land dedicated to avocado production...
1412,but tons is n t enough he said looking forwar...,but tons is n t enough iragorri said looking ...,but tons is n t enough he said looking forwar...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [ ]:
def pad_truncate(name):
  extra_length = 4
  name_length = len(name) - extra_length
  if name_length < 156:
    difference = 156 - name_length
    result = name + [tokenizer.eos_token_id] * difference
  elif name_length > 156:
     result = name[:156 + 3]+[tokenizer.eos_token_id]
  else:
    result = name
  return result

In [ ]:
import torch
def preprocess_data(data_to_process):
  torch.cuda.empty_cache()
  encode = tokenizer.encode(data_to_process + tokenizer.eos_token)
  padded = pad_truncate(encode)

  return  padded

In [ ]:
tokenize_train_data = train_data['training'].apply(preprocess_data)
tokenize_train_data

0         [30393, 7228, 45630, 504, 12876, 351, 1466, 28...
1         [30393, 7228, 45630, 504, 12876, 351, 1466, 28...
2         [25878, 651, 262, 12876, 284, 1907, 287, 5249,...
3         [86, 945, 286, 262, 2003, 481, 307, 8350, 351,...
4         [86, 945, 286, 262, 2003, 481, 307, 8350, 351,...
                                ...                        
102706    [896, 50207, 389, 517, 43180, 621, 584, 11676,...
102707    [896, 50207, 389, 517, 43180, 621, 584, 11676,...
102708    [1169, 7888, 7679, 393, 23644, 326, 36550, 262...
102709    [1169, 7888, 7679, 393, 23644, 326, 36550, 262...
102710    [896, 50207, 389, 517, 43180, 621, 584, 11676,...
Name: training, Length: 102711, dtype: object

In [ ]:
for item in tokenize_train_data:
  print(tokenizer.decode(item)), print(item)
  break

poll finds americans ok with women in combat SIMPLE: women get the ok to fight in combat units public approves<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endo

In [ ]:
tokenize_test_data = test_data['training'].apply(preprocess_data)
tokenize_test_data

0       [12518, 262, 9720, 318, 16968, 2029, 7370, 269...
1       [12518, 262, 9720, 318, 16968, 2029, 7370, 269...
2       [12518, 262, 9720, 318, 16968, 2029, 7370, 269...
3       [12518, 262, 9720, 3011, 845, 3024, 290, 262, ...
4       [12518, 262, 9720, 3011, 845, 3024, 2029, 7370...
                              ...                        
1409    [5661, 1285, 334, 13, 82, 13, 14791, 284, 951,...
1410    [392, 262, 1499, 318, 319, 2610, 284, 4439, 10...
1411    [4033, 2381, 666, 1956, 7256, 284, 40377, 3227...
1412    [475, 10860, 318, 299, 256, 1576, 339, 531, 20...
1413    [475, 10860, 318, 299, 256, 1576, 4173, 363, 2...
Name: training, Length: 1414, dtype: object

In [ ]:
from transformers import AutoModelWithLMHead
model = AutoModelWithLMHead.from_pretrained(model_checkpoints)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
metric = load_metric('rouge')

<ipython-input-17-10e4a614ebd6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
import pandas as pd

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    'gpt2-text-simplification_1e4_adafactor_newsela', #save directory
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    # gradient_accumulation_steps=4,
    # weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=20,
    # predict_with_generate=True,
    # eval_accumulation_steps=3,
    fp16=True, #available only with CUDA
    optim="adafactor",
    push_to_hub=True,
    # push_to_hub=True,
    )

trainer = Trainer(
    model,
    args,
    train_dataset=tokenize_train_data,
    eval_dataset=tokenize_test_data,
    data_collator=collator,
    tokenizer=tokenizer,
    # compute_metrics=torch_learn.compute_metrics,
    # compute_metrics=compute_rouge
)

/content/gpt2-text-simplification_1e4_adafactor_newsela is already a clone of https://huggingface.co/Smaraa/gpt2-text-simplification_1e4_adafactor_newsela. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.766200,0.875685
2,0.653800,0.901946
3,0.566300,0.955446
4,0.496100,0.999008
5,0.429900,1.027067
6,0.385300,1.054724
7,0.348200,1.109039
8,0.315200,1.138718
9,0.290300,1.185283
10,0.265500,1.208835


TrainOutput(global_step=32100, training_loss=0.33255749057757894, metrics={'train_runtime': 8665.1519, 'train_samples_per_second': 237.067, 'train_steps_per_second': 3.704, 'total_flos': 1.677347868672e+17, 'train_loss': 0.33255749057757894, 'epoch': 20.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/475M [00:00<?, ?B/s]

Upload file runs/Jun25_13-18-23_7caa027cf492/events.out.tfevents.1687699111.7caa027cf492.23401.0:   0%|       …

To https://huggingface.co/Smaraa/gpt2-text-simplification_1e4_adafactor_newsela
   c7849c4..da60509  main -> main

   c7849c4..da60509  main -> main

To https://huggingface.co/Smaraa/gpt2-text-simplification_1e4_adafactor_newsela
   da60509..5a864a8  main -> main

   da60509..5a864a8  main -> main



'https://huggingface.co/Smaraa/gpt2-text-simplification_1e4_adafactor_newsela/commit/da60509f8d32cdad95dea1aa8a72a3a444382971'

In [ ]:
!git clone https://github.com/feralvam/easse.git && cd easse && pip install .

Cloning into 'easse'...
remote: Enumerating objects: 1960, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1960 (delta 116), reused 102 (delta 102), pack-reused 1819
Receiving objects: 100% (1960/1960), 33.15 MiB | 18.16 MiB/s, done.
Resolving deltas: 100% (1229/1229), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/easse
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-0ldnj5fv/tseval_74152e6229254cd59f3b7df7a3e65c54
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-0ldnj5fv/tseval_74152e6229254cd59f3b7df7a3e65c54
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit f335e2e27026321c7c3d1dd63857416c7e73

In [ ]:
!cd easse && pip install .

In [ ]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.4 MB/s eta 0:00:00


In [ ]:
!cd easse && pip install .

In [ ]:
import os
import pandas as pd
from easse.sari import corpus_sari
from sacrebleu import corpus_bleu
import textstat

def evall(dataset, id):
  predictions = []
  actuals = []
  ref = []

  for _, sentence in dataset.iterrows():
    model_inputs = tokenizer(sentence['Normal'], return_tensors="pt")
    # print(model_inputs[0])
    # model_inputs = pad_truncate(model_inputs)
    generated_ids = model.generate(
              input_ids = model_inputs['input_ids'].to("cuda"),
              attention_mask = model_inputs['attention_mask'].to("cuda"),
              max_length=80,
              num_beams=4,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True,
              max_new_tokens=80
              )

    rez = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    predictions.extend(rez)
    actuals.append(sentence['Normal'])
    ref.append(sentence['Simple'])

  final_df = pd.DataFrame({"Actual Text": actuals, "Reff Text": ref, "Generated Text": predictions})
  final_df.to_csv(os.path.join("outputs", "predictions_" + str(id) + ".csv"))
  final_df['Generated Text'] =  [seq if ' SIMPLE: ' not in seq else seq.split(' SIMPLE: ')[1] for seq in list(final_df['Generated Text'])]

  eval = pd.DataFrame()
  eval['sari']=[corpus_sari(orig_sents=final_df['Actual Text'],
            sys_sents=final_df['Generated Text'],
            refs_sents=[final_df['Reff Text']])]
  eval['blue']=corpus_bleu(final_df['Generated Text'].tolist(), [final_df['Reff Text'].tolist()]).score
  count = 0
  for index, row in final_df.iterrows():
      if row['Generated Text'] == row['Actual Text']:
        count+=1

  eval['dublicates'] = count
  fkgl_scores = [textstat.flesch_kincaid_grade(text) for text in final_df['Generated Text']]
  eval["fkgl"] = sum(fkgl_scores) / len(fkgl_scores)

  eval.to_csv(os.path.join("outputs", "eval_" + str(id) + ".csv"))


In [ ]:
evall(test_data, 0)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=80) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=80) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=80) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`